# 搭建一个感知机_基于pytorch
先搭建一个线性系统，只有一层，没有激活函数


In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

net = nn.Sequential(nn.Linear(3,1))

# 定义损失失函数和优化器

criterion = nn.MSELoss()
optimizer1 = optim.SGD(net.parameters(), lr=0.001)

# 定义一个随机数据集

x = torch.randn(1000,3)


# 数据标准化

x_mean = x.mean(dim=0)
x_std = x.std(dim=0)
x = (x - x_mean) / x_std

y =  x[:, 0] * 5  + x[:, 1] * 6 + x[:, 2] * 7 + 3
print(x[0:3,:], y[0:3])
# 创建数据加载器
# 这个很有用，没有这个怎么都降不下去
dataset = TensorDataset(x, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

epoch = 100
for i in range(epoch):
    for input, result in dataloader:
    # 前向传播
        loss = criterion(net(input), result.unsqueeze(1))
        # 清除梯度函数，重新计算
        optimizer1.zero_grad()
        # 计算梯度
        loss.backward()
        # 更新参数
        optimizer1.step()
    if (i + 1) % 10 == 0:
        print(f'Epoch [{i+1}/{epoch}], Loss: {loss.sum():.4f}')

print(net.state_dict())


tensor([[ 0.2088,  0.8036, -0.1728],
        [-1.2670,  0.0211, -0.0956],
        [-0.7023,  0.0057,  2.4084]]) tensor([ 7.6557, -3.8775, 16.3815])
Epoch [10/100], Loss: 35.5390
Epoch [20/100], Loss: 4.0825
Epoch [30/100], Loss: 2.1204
Epoch [40/100], Loss: 0.4536
Epoch [50/100], Loss: 0.4009
Epoch [60/100], Loss: 0.0245
Epoch [70/100], Loss: 0.0352
Epoch [80/100], Loss: 0.0032
Epoch [90/100], Loss: 0.0043
Epoch [100/100], Loss: 0.0004
OrderedDict([('0.weight', tensor([[4.9890, 5.9873, 6.9859]])), ('0.bias', tensor([2.9957]))])


## softmax分类
用来对图像进行分类
1. 交叉熵损失函数
2. 数据的组织


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 数据相关
import torchvision


# 模型，损失函数，  优化器 Flatten用于将二维的数据转换为一维
# model = nn.Sequential(nn.Flatten(), nn.Linear(28*28, 256),nn.ReLU(),nn.Linear(256, 10),nn.Softmax())
model = nn.Sequential(nn.Flatten(), nn.Linear(28*28, 10))
model3 = nn.Sequential(nn.Flatten(), nn.Linear(28*28, 256), nn.BatchNorm1d(256), nn.ReLU(),nn.Linear(256, 10))
# 会自动进行softmax操作
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# 引入FashionMNIST数据集
# 数据集有2个维度，第一个为数据，第二个标签
train_data = torchvision.datasets.FashionMNIST(root='D:/data/FashionMNIST', train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_data = torchvision.datasets.FashionMNIST(root='D:/data/FashionMNIST', train=False, transform=torchvision.transforms.ToTensor(), download=True)

print(len(train_data))
print(len(test_data))

# 转为dataloader

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

def get_fashion_mnist_labels(labels): 
    """返回Fashion-MNIST数据集的文本标签"""
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
    'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]


60000
10000


In [5]:
i = 0
for m in model.modules():
    print(i,"-----------------------")
    print(m)
    i += 1

0 -----------------------
Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
)
1 -----------------------
Flatten(start_dim=1, end_dim=-1)
2 -----------------------
Linear(in_features=784, out_features=10, bias=True)


: 

In [53]:



# 开始训练
epochs = 50

for epoch in range(epochs):
    train_loss = 0
    train_acc = 0
    model.train()  # 训练模式
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer.step()
    print(f'epoch : {epoch}/{epochs}, loss : {train_loss / len(train_loader)}')

epoch : 0/50, loss : 1.6781267035744591
epoch : 1/50, loss : 1.1892392269965173
epoch : 2/50, loss : 1.0157622687979293
epoch : 3/50, loss : 0.9245630260596651
epoch : 4/50, loss : 0.8659877585830973
epoch : 5/50, loss : 0.8242006770202092
epoch : 6/50, loss : 0.7922710006170944
epoch : 7/50, loss : 0.7666691566454068
epoch : 8/50, loss : 0.7453503385662779
epoch : 9/50, loss : 0.7275450648402354
epoch : 10/50, loss : 0.7120632321468549
epoch : 11/50, loss : 0.6985825799040194
epoch : 12/50, loss : 0.6864457185715754
epoch : 13/50, loss : 0.6758444571990703
epoch : 14/50, loss : 0.6661140362718212
epoch : 15/50, loss : 0.6573320093757308
epoch : 16/50, loss : 0.6491434088965723
epoch : 17/50, loss : 0.6418653217587136
epoch : 18/50, loss : 0.6350175707197901
epoch : 19/50, loss : 0.628741068531201
epoch : 20/50, loss : 0.622972018111235
epoch : 21/50, loss : 0.6173229749078182
epoch : 22/50, loss : 0.6120180882561181
epoch : 23/50, loss : 0.6074027686929906
epoch : 24/50, loss : 0.6026

In [57]:
# 测试
model.eval()
acc = 0
for i, (inputs, labels) in enumerate(test_loader) :
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    result = preds == labels
    acc += torch.sum(result).item()

print(f'Test Accuracy : {acc / len(test_loader.dataset)}')

Test Accuracy : 0.813


## 多层感知机分类
1. 3层网络进行分类，数据不变

In [58]:
model2 = nn.Sequential(nn.Flatten(), nn.Linear(28*28, 256),nn.ReLU(),nn.Linear(256, 10))
optimizer2 = optim.SGD(model2.parameters(), lr=0.001)
# 开始训练
epochs = 50

for epoch in range(epochs):
    train_loss = 0
    train_acc = 0
    model2.train()  # 训练模式
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer2.zero_grad()
        # 前向传播
        outputs = model2(inputs)
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer2.step()
    print(f'epoch : {epoch}/{epochs}, loss : {train_loss / len(train_loader)}')

epoch : 0/50, loss : 2.058189784031687
epoch : 1/50, loss : 1.6116484215518814
epoch : 2/50, loss : 1.2809890359957843
epoch : 3/50, loss : 1.0900558887132958
epoch : 4/50, loss : 0.9759939175043533
epoch : 5/50, loss : 0.9011643505426866
epoch : 6/50, loss : 0.8478422816882509
epoch : 7/50, loss : 0.8073097468693373
epoch : 8/50, loss : 0.7750162648112535
epoch : 9/50, loss : 0.748060812192685
epoch : 10/50, loss : 0.725067481176177
epoch : 11/50, loss : 0.7049987691361258
epoch : 12/50, loss : 0.6871420713757147
epoch : 13/50, loss : 0.6712344519492152
epoch : 14/50, loss : 0.6568835789460872
epoch : 15/50, loss : 0.6438381469198889
epoch : 16/50, loss : 0.6320331890318693
epoch : 17/50, loss : 0.6211585829824781
epoch : 18/50, loss : 0.6112891066747942
epoch : 19/50, loss : 0.6021736372572019
epoch : 20/50, loss : 0.5935407889041819
epoch : 21/50, loss : 0.5858222726883411
epoch : 22/50, loss : 0.5785395826485111
epoch : 23/50, loss : 0.5716355844919108
epoch : 24/50, loss : 0.56542

In [60]:
epochs = 50

for epoch in range(epochs):
    train_loss = 0
    train_acc = 0
    model2.train()  # 训练模式
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer2.zero_grad()
        # 前向传播
        outputs = model2(inputs)
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer2.step()
    print(f'epoch : {epoch}/{epochs}, loss : {train_loss / len(train_loader)}')

epoch : 0/50, loss : 0.47996885531238404
epoch : 1/50, loss : 0.478011869021189
epoch : 2/50, loss : 0.47634257963979676
epoch : 3/50, loss : 0.4746877691511915
epoch : 4/50, loss : 0.4730023661179583
epoch : 5/50, loss : 0.47128506509988294
epoch : 6/50, loss : 0.4697705915710057
epoch : 7/50, loss : 0.46838838937503696
epoch : 8/50, loss : 0.4668065632330075
epoch : 9/50, loss : 0.46533231313294693
epoch : 10/50, loss : 0.46397538970846103
epoch : 11/50, loss : 0.46254531411664573
epoch : 12/50, loss : 0.4615260833330246
epoch : 13/50, loss : 0.4599114564150127
epoch : 14/50, loss : 0.4585916258251743
epoch : 15/50, loss : 0.4573958822722628
epoch : 16/50, loss : 0.4561487347332399
epoch : 17/50, loss : 0.45501063784747237
epoch : 18/50, loss : 0.4536895352258865
epoch : 19/50, loss : 0.45278060798451847
epoch : 20/50, loss : 0.4515769419226565
epoch : 21/50, loss : 0.4503793278768627
epoch : 22/50, loss : 0.4492086807547856
epoch : 23/50, loss : 0.4486123515662354
epoch : 24/50, los

In [61]:
# 测试
model2.eval()
acc = 0
for i, (inputs, labels) in enumerate(test_loader) :
    outputs = model2(inputs)
    _, preds = torch.max(outputs, 1)
    result = preds == labels
    acc += torch.sum(result).item()

print(f'Test Accuracy : {acc / len(test_loader.dataset)}')

Test Accuracy : 0.8371


## 优化
加入批量归一化的的感知机

In [5]:
optimizer3 = optim.SGD(model3.parameters(), lr=0.001)
# 开始训练
epochs = 100

for epoch in range(epochs):
    train_loss = 0
    train_acc = 0
    model3.train()  # 训练模式
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer3.zero_grad()
        # 前向传播
        outputs = model3(inputs)
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer3.step()
    print(f'mode3 epoch : {epoch + 1}/{epochs}, loss : {train_loss / len(train_loader)}')

mode3 epoch : 1/100, loss : 0.6751904030090202
mode3 epoch : 2/100, loss : 0.6009824399564312
mode3 epoch : 3/100, loss : 0.5569122223012737
mode3 epoch : 4/100, loss : 0.5255958432518343
mode3 epoch : 5/100, loss : 0.5042006560861428
mode3 epoch : 6/100, loss : 0.48724014610687555
mode3 epoch : 7/100, loss : 0.472957036197821
mode3 epoch : 8/100, loss : 0.4604240874650636
mode3 epoch : 9/100, loss : 0.44890757905903145
mode3 epoch : 10/100, loss : 0.44151464927552353
mode3 epoch : 11/100, loss : 0.43361444043706476
mode3 epoch : 12/100, loss : 0.42670734492001505
mode3 epoch : 13/100, loss : 0.4193474128524632
mode3 epoch : 14/100, loss : 0.41432717852373874
mode3 epoch : 15/100, loss : 0.40957382364250194
mode3 epoch : 16/100, loss : 0.4030560329556465
mode3 epoch : 17/100, loss : 0.398170730817928
mode3 epoch : 18/100, loss : 0.39309568399750094
mode3 epoch : 19/100, loss : 0.38901904973584706
mode3 epoch : 20/100, loss : 0.38444786879426635
mode3 epoch : 21/100, loss : 0.3811117032

In [6]:
# 测试
model3.eval()
acc = 0
for i, (inputs, labels) in enumerate(test_loader) :
    outputs = model3(inputs)
    _, preds = torch.max(outputs, 1)
    result = preds == labels
    acc += torch.sum(result).item()

print(f'model3 Test Accuracy : {acc / len(test_loader.dataset)}')

model3 Test Accuracy : 0.8855


## 卷积神经网络

In [34]:
# 卷积层，激活函数，卷积层，激活函数， 展开层，softmax
# nn.Flatten(), nn.Linear(28*28, 256),nn.ReLU(),
# model3 = nn.Sequential(nn.Flatten(), nn.Linear(28*28, 256), nn.BatchNorm1d(256), nn.ReLU(),nn.Linear(256, 10))
model4 = nn.Sequential(nn.Conv2d(1,10, kernel_size=5,padding=2), nn.BatchNorm2d(10), 
                       nn.ReLU(), nn.Conv2d(10,10,kernel_size=7,padding=3,stride=2), 
                       nn.BatchNorm2d(10), nn.ReLU(), nn.AvgPool2d(kernel_size=2, stride=2),
                       nn.Flatten(),nn.Linear(490,10))

In [35]:
optimizer4 = optim.SGD(model4.parameters(), lr=0.1)
# 开始训练
epochs = 20

for epoch in range(epochs):
    train_loss = 0
    train_acc = 0
    model4.train()  # 训练模式
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer4.zero_grad()
        # 前向传播
        outputs = model4(inputs)
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer4.step()
    print(f'mode4 epoch : {epoch + 1}/{epochs}, loss : {train_loss / len(train_loader)}')

mode4 epoch : 1/20, loss : 0.46989872158844587
mode4 epoch : 2/20, loss : 0.33593049359474103
mode4 epoch : 3/20, loss : 0.3028152750721618
mode4 epoch : 4/20, loss : 0.28459284491916453
mode4 epoch : 5/20, loss : 0.2705826828760633
mode4 epoch : 6/20, loss : 0.26009566962385355
mode4 epoch : 7/20, loss : 0.25164625094707077
mode4 epoch : 8/20, loss : 0.24447247807754638
mode4 epoch : 9/20, loss : 0.23814794306418915
mode4 epoch : 10/20, loss : 0.23332662656029532
mode4 epoch : 11/20, loss : 0.22762316469746485
mode4 epoch : 12/20, loss : 0.2239819091718906
mode4 epoch : 13/20, loss : 0.22040698805184505
mode4 epoch : 14/20, loss : 0.21534072336897667
mode4 epoch : 15/20, loss : 0.21301131394864525
mode4 epoch : 16/20, loss : 0.20845385730218913
mode4 epoch : 17/20, loss : 0.2061681342023268
mode4 epoch : 18/20, loss : 0.20345852216248955
mode4 epoch : 19/20, loss : 0.20078534877964302
mode4 epoch : 20/20, loss : 0.1978468952188169


In [36]:
# 测试
model4.eval()
acc = 0
for i, (inputs, labels) in enumerate(test_loader) :
    outputs = model4(inputs)
    _, preds = torch.max(outputs, 1)
    result = preds == labels
    acc += torch.sum(result).item()

print(f'model4 Test Accuracy : {acc / len(test_loader.dataset)}')

acc = 0
for i, (inputs, labels) in enumerate(train_loader) :
    outputs = model4(inputs)
    _, preds = torch.max(outputs, 1)
    result = preds == labels
    acc += torch.sum(result).item()

print(f'model4 train Accuracy : {acc / len(train_loader.dataset)}')

model4 Test Accuracy : 0.9006
model4 train Accuracy : 0.92965


In [14]:
model5 = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10))

In [27]:
optimizer5 = optim.SGD(model5.parameters(), lr=0.1)
# 开始训练
epochs = 10

for epoch in range(epochs):
    train_loss = 0
    train_acc = 0
    model5.train()  # 训练模式
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer5.zero_grad()
        # 前向传播
        outputs = model5(inputs)
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer5.step()
    print(f'mode5 epoch : {epoch + 1}/{epochs}, loss : {train_loss / len(train_loader)}')

mode5 epoch : 1/10, loss : 0.23562729865042512
mode5 epoch : 2/10, loss : 0.23490782733013763
mode5 epoch : 3/10, loss : 0.23406693687253416
mode5 epoch : 4/10, loss : 0.23300409369837882
mode5 epoch : 5/10, loss : 0.23249410330327844
mode5 epoch : 6/10, loss : 0.23177548307282075
mode5 epoch : 7/10, loss : 0.23105357801799836
mode5 epoch : 8/10, loss : 0.23039482366730538
mode5 epoch : 9/10, loss : 0.2293452197936044
mode5 epoch : 10/10, loss : 0.22856990294034546


In [28]:
# 测试
model5.eval()
acc = 0
for i, (inputs, labels) in enumerate(test_loader) :
    outputs = model5(inputs)
    _, preds = torch.max(outputs, 1)
    result = preds == labels
    acc += torch.sum(result).item()

print(f'model5 Test Accuracy : {acc / len(test_loader.dataset)}')

acc = 0
for i, (inputs, labels) in enumerate(train_loader) :
    outputs = model5(inputs)
    _, preds = torch.max(outputs, 1)
    result = preds == labels
    acc += torch.sum(result).item()

print(f'model5 train Accuracy : {acc / len(train_loader.dataset)}')
# model5 Test Accuracy : 0.8957
# model5 train Accuracy : 0.9139166666666667

model5 Test Accuracy : 0.898
model5 train Accuracy : 0.9161666666666667
